# Runtime 运行时

LangGraph 暴露了一个 Runtime 对象，包含以下信息：
- 上下文 : 静态信息，如用户 ID、数据库连接或其他代理调用的依赖项
- 存储 : 一个 BaseStore 实例，用于 长期记忆
- 流写入器 : 一个用于通过 "custom" 流模式传输信息的对象


In [11]:
from dataclasses import dataclass

from langchain.agents import create_agent

# 官方示例有误，这样定义只会在 Context 保留 user_name 属性, 而模型不知道用户叫什么名字

@dataclass
class Context:
    user_name: str

agent = create_agent(
    model="gpt-5-nano",
    tools=[...],
    context_schema=Context  
)

agent.invoke(
    {"messages": [{"role": "user", "content": "What's my name?"}]},
    context=Context(user_name="John Smith")  
)

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I don’t know your name yet. What would you like me to call you? By the way, I’m John Smith, your assistant.


## 内部工具

In [15]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime  
from langchain.agents import create_agent
@dataclass
class Context:
    user_id: str

@tool
def fetch_user_email_preferences(runtime: ToolRuntime[Context]) -> str:  
    """Fetch the user's email preferences from the store."""
    user_id = runtime.context.user_id  

    preferences: str = "这位用户希望你写一封简短而礼貌的电子邮件。"
    if runtime.store:  
        if memory := runtime.store.get(("users",), user_id):  
            preferences = memory.value["preferences"]

    return preferences

agent = create_agent(
    model="gpt-5-nano",
    tools=[fetch_user_email_preferences],
    context_schema=Context
)

# 模拟运行
result = agent.invoke(
    {"messages": [{"role": "user", "content": "帮我写封邮件"}]},
    context=Context(user_id="u1234")
)

for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

帮我写封邮件
================================== Ai Message ==================================

当然可以。为了给你写出最合适的邮件，我需要了解一些信息。你可以先告诉我：
- 收件人与场景（如商务合作、求职应聘、项目汇报、会议邀请等）
- 邮件语气偏好（正式/半正式/友好）
- 邮件长度偏好（简短/详细）
- 是否需要附上附件（如简历、报告、资料等）
- 需要突出的重点信息有哪些

如果你要先用一个可直接用的模板，下面给你四个常见场景的可直接使用版本，你只需把方括号里的信息替换成实际内容即可。

1) 商务合作/咨询（正式、简短）
Subject: [事项/合作意向] 询问与合作建议
尊敬的 [姓名/职位]，您好。我是 [你的姓名]，来自 [公司名]，负责 [领域/项目]。现就 [具体事项] 想了解贵方的 [需求/条件]。若方便，请告知贵方的意向与可行的时间安排。感谢您的时间，期待沟通。祝好，  
[你的姓名] | [职位] | [公司] | [电话/邮箱]

2) 求职应聘邮件（正式、清晰）
Subject: 应聘 [职位] - [姓名]
尊敬的 [招聘负责人/先生/女士]，您好。 我叫 [姓名]，毕业于 [学校/专业]，拥有 [X] 年相关经验，现申请贵司的 [职位]。在 [前任公司/经历]，我负责 [职责]，取得 [成果]。附上我的简历与作品集，恳请您审阅。若需要我提供更多信息，愿意随时配合。感谢您的时间，期待有机会进一步交流。此致，  
[姓名] | [手机号] | [邮箱]  
附件：简历、作品集

3) 项目进展汇报（正式、详细）
Subject: 项目 [项目名] 进展汇报（日期）
尊敬的 [姓名]，您好。现汇报 [项目名] 的最新进展、里程碑及后续计划。要点如下：  
- 已完成：  
- 进行中：  
- 风险与应对：  
- 下一步计划：  
如需，我可以附上详细进度表与风险评估，请您审阅。感谢您的关注，期待您的意见或指示。此致，  
[姓名] | [职位] | [公司] | [联系方式]

4) 会议邀请

## 内部中间件

In [2]:
from dataclasses import dataclass

from langchain.messages import AnyMessage
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import dynamic_prompt, ModelRequest, before_model, after_model
from langgraph.runtime import Runtime


@dataclass
class Context:
    user_name: str

# Dynamic prompts
@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    user_name = request.runtime.context.user_name  
    system_prompt = f"You are a helpful assistant. Address the user as {user_name}."
    return system_prompt

# Before model hook
@before_model
def log_before_model(state: AgentState, runtime: Runtime[Context]) -> dict | None:  
    print(f"Processing request for user: {runtime.context.user_name}")  
    return None

# After model hook
@after_model
def log_after_model(state: AgentState, runtime: Runtime[Context]) -> dict | None:  
    print(f"Completed request for user: {runtime.context.user_name}")  
    return None

agent = create_agent(
    model="gpt-5-nano",
    tools=[],
    middleware=[dynamic_system_prompt, log_before_model, log_after_model],  
    context_schema=Context
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my name?"}]},
    context=Context(user_name="John Smith")
)

for message in result["messages"]:
    message.pretty_print()

Processing request for user: John Smith
Completed request for user: John Smith
================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is John Smith.
